# 카카오 로컬 API 활용

In [1]:
import requests
from urllib.parse import quote
import pandas as pd

- API Key 가져오기

In [2]:
with open('kakaoapikey.txt') as file:
    kakao_key = file.read()
len(kakao_key)

32

- URL 만들기

In [3]:
local_url = 'https://dapi.kakao.com/v2/local/search/address.json'
addr = '서울특별시 서초구 남부순환로 2584(서초동)'
url = f'{local_url}?query={quote(addr)}'
url

'https://dapi.kakao.com/v2/local/search/address.json?query=%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C%20%EC%84%9C%EC%B4%88%EA%B5%AC%20%EB%82%A8%EB%B6%80%EC%88%9C%ED%99%98%EB%A1%9C%202584%28%EC%84%9C%EC%B4%88%EB%8F%99%29'

In [4]:
# Header: "Authorization: KakaoAK ${REST_API_KEY}"
header = {'Authorization': f'KakaoAK {kakao_key}'}

In [5]:
result = requests.get(url, headers=header).json()
result

{'documents': [{'address': {'address_name': '서울 서초구 서초동 1376-3',
    'b_code': '1165010800',
    'h_code': '1165052000',
    'main_address_no': '1376',
    'mountain_yn': 'N',
    'region_1depth_name': '서울',
    'region_2depth_name': '서초구',
    'region_3depth_h_name': '서초2동',
    'region_3depth_name': '서초동',
    'sub_address_no': '3',
    'x': '127.032683002019',
    'y': '37.4836248649455'},
   'address_name': '서울 서초구 남부순환로 2584',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '서울 서초구 남부순환로 2584',
    'building_name': '서초구청, 서초구의회',
    'main_building_no': '2584',
    'region_1depth_name': '서울',
    'region_2depth_name': '서초구',
    'region_3depth_name': '서초동',
    'road_name': '남부순환로',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '127.032683002019',
    'y': '37.4836248649455',
    'zone_no': '06750'},
   'x': '127.032683002019',
   'y': '37.4836248649455'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}